In [1]:
%reset -fs
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import time, os
from bs4 import BeautifulSoup
import datetime
import re
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [2]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [3]:
driver = webdriver.Chrome(chromedriver)

In [4]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/'
query = '?date=' + datetime.date(2021, 10, 19).strftime('%Y-%m-%d')
bet_query = bet_search + query

In [5]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [6]:
bet_query = bet_search + '?date=' + '2021-12-20'
driver.get(bet_query)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [8]:
# Tip of the hat to Rita for helping with these two functions !
def get_site_rb(date):
    bet_query = bet_search + '?date=' + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [9]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = final_df.append(pd.DataFrame(data))
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df


In [10]:
#run this and then close the cookies pop-up!
for i in date_list:
    
    if i == '2021-12-24':
        continue
    if i == '2021-11-25':
        continue
    get_site_rb(i)

In [11]:
driver.close()

In [12]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [13]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [14]:
final_df.reset_index(inplace=True, drop=True)

In [15]:
final_df.drop(columns=['index'], inplace=True)

KeyError: "['index'] not found in axis"

In [ ]:
team = final_df.Team.str.split(" ", expand = True)

In [ ]:
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [ ]:
opp = final_df.Opp.str.split(" ", expand = True)

In [ ]:
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [ ]:
bet_info = final_df.Line.str.split(" ", expand = True)

In [ ]:
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [ ]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [ ]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [ ]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [ ]:
final_df['seven_day_accuracy'] = accuracies[8]

In [ ]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [ ]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [ ]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [ ]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [ ]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [ ]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [ ]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [ ]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [ ]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [ ]:
sorted_df = sorted_df.replace('TBD', np.nan)

In [ ]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [ ]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x));sorted_df

In [ ]:
results = sorted_df.Result.str.split(' ', expand=True);

In [ ]:
sorted_df.Result = results[5]

In [ ]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [ ]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u'))
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0']

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [ ]:
sorted_df.to_csv('betting_data.csv')